In [1]:
from data_generator import *
from init_sdr import *
from main import *
from common import *
from measure import *

from cv_model import *
from test_model import *

In [2]:
import json

import ray
import time
import warnings

import torch
import argparse
import ray
from ray import tune
from ray.tune import Trainable
from itertools import product
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import datasets, transforms

import ray
import copy


import time

from sklearn.model_selection import KFold

import argparse

import shutil

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search import ConcurrencyLimiter
from typing import List, Dict

from ray.tune import CLIReporter
import warnings

from ray.tune.search.bayesopt import BayesOptSearch


import math

In [3]:
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import HyperBandScheduler
import shutil

## 일단은 Xaiver로

In [4]:
import time
start_time = time.time()


ray.init(ignore_reinit_error=True)

warnings.filterwarnings("ignore", category=UserWarning)


num_var = 20


hyper_space = {"h_dim1": int(0.5 * num_var),
             'h_dim2': num_var ,
             "sample_size": 500,
             "num_features": num_var,
             "lambda_0":  tune.grid_search([1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1.0, 2.0]),
             'lambda_1': tune.grid_search([1e-1, 2.5e-1, 5e-1, 7.5e-1, 1.0, 1.5 ,2.0 ,2.5,3.0]),
             'lr':  tune.grid_search([1e-4, 2.5e-4, 5e-4, 7.5e-4, 1e-3, 2.5e-3, 5e-3, 7.5e-3, 1e-2]),
             'epoch':  100,
             'seed': -1,
             "model_num": 'model1',
             "num_cv": 5,
             "or_pen": tune.grid_search(['pro', 'orthogonal']),
             "dim" : 'unknown'
}


local_dir = 'cv_0419/'
exp_name = "bn_transform"

study1 = tune.run(
        cv_train_model,
        config=hyper_space, 
        resources_per_trial={"cpu": 2}, 
        metric = 'cv_custom_loss' ,
        mode = 'min',
        raise_on_failed_trial=False, 
        verbose =  False 
)

best_config1 = study1.get_best_config(metric="cv_custom_loss", mode = 'min')

ray.shutdown()

end_time = time.time()
execution_time = (end_time - start_time) / 60

print("Execution time:", execution_time, "minutes")
print('-' * 200)
print(best_config1)

(cv_train_model pid=356633) lambda0: 0.01 lambda_1 1.0 lr:  0.0001 h_dim2: 20 Epochs 100 est_dim:  10.0 or_pen pro
(cv_train_model pid=356633) cv_val_mse: 37.8784481048584 
(cv_train_model pid=356633)  cv_custom_loss 49.630273056030276
(cv_train_model pid=356630) lambda0: 2.0 lambda_1 0.75 lr:  0.0001 h_dim2: 20 
(cv_train_model pid=356630) Epochs 100 est_dim: 
(cv_train_model pid=356630)  10.0 
(cv_train_model pid=356630) or_pen pro
(cv_train_model pid=356630) 
(cv_train_model pid=356630) 37.92021160125732 
(cv_train_model pid=356630) 
(cv_train_model pid=356603) lambda0: 0.01 lambda_1 2.5 lr:  0.0001 h_dim2: 20 Epochs 100 est_dim:  10.0 or_pen pro [repeated 56x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(cv_train_model pid=356603) cv_val_mse: 37.60821762084961  [repeated 57x across cluster]
(cv_train_model pid=356603) 

In [5]:
print(best_config1)

{'h_dim1': 10, 'h_dim2': 20, 'sample_size': 500, 'num_features': 20, 'lambda_0': 0.005, 'lambda_1': 0.1, 'lr': 0.01, 'epoch': 100, 'seed': -1, 'model_num': 'model1', 'num_cv': 5, 'or_pen': 'orthogonal', 'dim': 'unknown'}


In [18]:
import time
start_time = time.time()


ray.init(ignore_reinit_error=True)

warnings.filterwarnings("ignore", category=UserWarning)

num_var = 20

hyper_space = {"h_dim1":1,
             'h_dim2': num_var ,
             "sample_size": 1000,
             "num_features": num_var,
             "lambda_0":  tune.grid_search([1e-3, 5e-3,0.01, 0.05, 0.07, 0.1, 0.5]),
             'lambda_1': tune.grid_search([0.1]),
             'lr':  tune.grid_search([1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]),
             'epoch':  1000,
             'seed': -1,
             "model_num": 'model4',
             "num_cv": 5,
             "or_pen": tune.grid_search(['pro', 'orthogonal']),
             "dim" : 'known'
}

local_dir = 'cv_0419/'
exp_name = "bn_transform"

study1 = tune.run(
        cv_train_model,
        config=hyper_space, 
        resources_per_trial={"cpu": 32}, 
        metric = 'cv_custom_loss' ,
        mode = 'min',
        raise_on_failed_trial=False, 
        verbose =  False 
)

best_config2 = study1.get_best_config(metric="cv_custom_loss", mode = 'min')

ray.shutdown()

end_time = time.time()
execution_time = (end_time - start_time) / 60

print("Execution time:", execution_time, "minutes")
print('-' * 200)
print(best_config2)

(cv_train_model pid=1248861) lambda0: 0.001 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248861) cv_val_mse: 1.849621868133545 
(cv_train_model pid=1248861)  cv_custom_loss 1.8534056186676025
(cv_train_model pid=1248865) lambda0: 0.005 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248865) cv_val_mse: 1.8506564378738404 
(cv_train_model pid=1248865)  cv_custom_loss 1.8701382637023927
(cv_train_model pid=1248866) lambda0: 0.01 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248866) cv_val_mse: 1.858734405040741 
(cv_train_model pid=1248866)  cv_custom_loss 1.8924262046813964
(cv_train_model pid=1248866) lambda0: 0.1 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248866) cv_val_mse: 1.8294134140014648 
(cv_train_model pid=1248866)  cv_custom_loss 2.1936015605926515
(cv_train_model pid=1248865) lambda0: 0.0